In [73]:
from dataclasses import dataclass
import lzma
import pickle
from functools import total_ordering

In [74]:
from sc2.game_info import GameInfo
from sc2.position import Point2
import numpy as np
import scipy
import plotly.express as px

In [75]:
np.random.seed(0)

In [76]:
import sys
sys.path.append("../bot")

from utils.dijkstra import shortest_paths_opt

In [77]:
%load_ext line_profiler
%load_ext Cython

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [78]:
with lzma.open("../resources/GresvanAIE.xz", "rb") as f:
    game_info: GameInfo = pickle.load(f)
game_info.players

[<sc2.player.Player at 0x1d06eaa7350>, <sc2.player.Player at 0x1d06eaa7290>]

In [79]:
%%cython
from cython cimport boundscheck, wraparound

cdef unsigned int euclidean_distance_squared_int((unsigned int, unsigned int) p1, (unsigned int, unsigned int) p2):
    return (p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2

cpdef set flood_fill(
    (unsigned int, unsigned int) start_point, 
    const unsigned char[:, :] terrain_grid, 
    const unsigned char[:, :] pathing_grid, 
    unsigned int max_distance, 
    set cutoff_points
):
    cdef:
        unsigned int terrain_height = terrain_grid[start_point[0], start_point[1]]
        unsigned int pathing_value = pathing_grid[start_point[0], start_point[1]]
        set filled_points = set()

    # Only continue if we can get a height for the starting point
    if not terrain_height:
        return filled_points
    
    if pathing_value != 1:
        return filled_points
        
    grid_flood_fill(start_point, terrain_grid, pathing_grid, terrain_height, filled_points, start_point, max_distance, cutoff_points)
    return filled_points

cdef set grid_flood_fill(
    (unsigned int, unsigned int) point, 
    const unsigned char[:, :] terrain_grid, 
    const unsigned char[:, :] pathing_grid, 
    unsigned int target_val, 
    set current_vec, 
    (unsigned int, unsigned int) start_point, 
    unsigned int max_distance, 
    set cutoff_points):
    cdef:
        unsigned int terrain_height = terrain_grid[start_point[0], start_point[1]]
        unsigned int pathing_value = pathing_grid[start_point[0], start_point[1]]
    # Check that we haven't already added this point.
    if point in current_vec:
        return current_vec

    # Check that this point isn't too far away from the start
    if euclidean_distance_squared_int(point, start_point) > max_distance ** 2:
        return current_vec

    if point in cutoff_points:
        return current_vec

    terrain_height = terrain_grid[point[0], point[1]]
    pathing_value = pathing_grid[point[0], point[1]]
    if terrain_height != target_val or pathing_value != 1:
        return current_vec
    
    current_vec.add(point)
    grid_flood_fill((point[0]+1, point[1]), terrain_grid, pathing_grid, terrain_height, current_vec, start_point, max_distance, cutoff_points)
    grid_flood_fill((point[0]-1, point[1]), terrain_grid, pathing_grid, terrain_height, current_vec, start_point, max_distance, cutoff_points)
    grid_flood_fill((point[0], point[1]+1), terrain_grid, pathing_grid, terrain_height, current_vec, start_point, max_distance, cutoff_points)
    grid_flood_fill((point[0], point[1]-1), terrain_grid, pathing_grid, terrain_height, current_vec, start_point, max_distance, cutoff_points)

In [97]:
%%cython --compile-args=-Ofast
# cython: language_level=3, boundscheck=False, wraparound=False, embedsignature=False, initializedcheck=False
# distutils: define_macros=NPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION

#include <functional>
#include <queue>

import numpy as np
import heapq
cimport numpy as cnp
from libc.stdlib cimport free, malloc
# data type for the key value
ctypedef cnp.float64_t DTYPE_t
cdef DTYPE_t DTYPE_INF = <DTYPE_t>np.finfo(dtype=np.float64).max
cdef enum ElementState:
   SCANNED     = 1     # popped from the heap
   NOT_IN_HEAP = 2     # never been in the heap
   IN_HEAP     = 3     # in the heap
cdef struct Element:
    ElementState state # element state wrt the heap
    size_t node_idx   # index of the corresponding node in the tree
    DTYPE_t key        # key value
cdef struct PriorityQueue:
    size_t  length    # maximum heap size
    size_t  size      # number of elements in the heap
    size_t* A         # array storing the binary tree
    Element* Elements  # array storing the elements
cdef void init_pqueue(
    PriorityQueue* pqueue,
    size_t length) noexcept nogil:
    """Initialize the priority queue.

    input
    =====
    * PriorityQueue* pqueue : priority queue
    * size_t length : length (maximum size) of the heap
    """
    cdef size_t i
    pqueue.length = length
    pqueue.size = 0
    pqueue.A = <size_t*> malloc(length * sizeof(size_t))
    pqueue.Elements = <Element*> malloc(length * sizeof(Element))
    for i in range(length):
        pqueue.A[i] = length
        _initialize_element(pqueue, i)
cdef inline void _initialize_element(
    PriorityQueue* pqueue,
    size_t element_idx) noexcept nogil:
    """Initialize a single element.

    input
    =====
    * PriorityQueue* pqueue : priority queue
    * size_t element_idx : index of the element in the element array
    """
    pqueue.Elements[element_idx].key = DTYPE_INF
    pqueue.Elements[element_idx].state = NOT_IN_HEAP
    pqueue.Elements[element_idx].node_idx = pqueue.length
cdef void free_pqueue(
    PriorityQueue* pqueue) noexcept nogil:
    """Free the priority queue.

    input
    =====
    * PriorityQueue* pqueue : priority queue
    """
    free(pqueue.A)
    free(pqueue.Elements)
cdef void insert(
    PriorityQueue* pqueue,
    size_t element_idx,
    DTYPE_t key) noexcept nogil:
    """Insert an element into the priority queue and reorder the heap.

    input
    =====
    * PriorityQueue* pqueue : priority queue
    * size_t element_idx : index of the element in the element array
    * DTYPE_t key : key value of the element

    assumptions
    ===========
    * the element pqueue.Elements[element_idx] is not in the heap
    * its new key is smaller than DTYPE_INF
    """
    cdef size_t node_idx = pqueue.size
    pqueue.size += 1
    pqueue.Elements[element_idx].state = IN_HEAP
    pqueue.Elements[element_idx].node_idx = node_idx
    pqueue.A[node_idx] = element_idx
    _decrease_key_from_node_index(pqueue, node_idx, key)
cdef void decrease_key(
    PriorityQueue* pqueue,
    size_t element_idx, 
    DTYPE_t key_new) noexcept nogil:
    """Decrease the key of a element in the priority queue, 
    given its element index.

    input
    =====
    * PriorityQueue* pqueue : priority queue
    * size_t element_idx : index of the element in the element array
    * DTYPE_t key_new : new value of the element key 

    assumption
    ==========
    * pqueue.Elements[idx] is in the heap
    """
    _decrease_key_from_node_index(
        pqueue, 
        pqueue.Elements[element_idx].node_idx, 
        key_new)
cdef size_t extract_min(PriorityQueue* pqueue) noexcept nogil:
    """Extract element with min key from the priority queue, 
    and return its element index.

    input
    =====
    * PriorityQueue* pqueue : priority queue

    output
    ======
    * size_t : element index with min key

    assumption
    ==========
    * pqueue.size > 0
    """
    cdef: 
        size_t element_idx = pqueue.A[0]  # min element index
        size_t node_idx = pqueue.size - 1  # last leaf node index
    # exchange the root node with the last leaf node
    _exchange_nodes(pqueue, 0, node_idx)
    # remove this element from the heap
    pqueue.Elements[element_idx].state = SCANNED
    pqueue.Elements[element_idx].node_idx = pqueue.length
    pqueue.A[node_idx] = pqueue.length
    pqueue.size -= 1
    # reorder the tree elements from the root node
    _min_heapify(pqueue, 0)
    return element_idx
cdef inline void _exchange_nodes(
    PriorityQueue* pqueue, 
    size_t node_i,
    size_t node_j) noexcept nogil:
    """Exchange two nodes in the heap.

    input
    =====
    * PriorityQueue* pqueue : priority queue
    * size_t node_i: first node index
    * size_t node_j: second node index
    """
    cdef: 
        size_t element_i = pqueue.A[node_i]
        size_t element_j = pqueue.A[node_j]
    
    # exchange element indices in the heap array
    pqueue.A[node_i] = element_j
    pqueue.A[node_j] = element_i
    # exchange node indices in the element array
    pqueue.Elements[element_j].node_idx = node_i
    pqueue.Elements[element_i].node_idx = node_j
    
cdef inline void _min_heapify(
    PriorityQueue* pqueue,
    size_t node_idx) noexcept nogil:
    """Re-order sub-tree under a given node (given its node index) 
    until it satisfies the heap property.

    input
    =====
    * PriorityQueue* pqueue : priority queue
    * size_t node_idx : node index
    """
    cdef: 
        size_t l, r, i = node_idx, s
    while True:
        l =  2 * i + 1  
        r = l + 1
        
        if (
            (l < pqueue.size) and 
            (pqueue.Elements[pqueue.A[l]].key < pqueue.Elements[pqueue.A[i]].key)
        ):
            s = l
        else:
            s = i
        if (
            (r < pqueue.size) and 
            (pqueue.Elements[pqueue.A[r]].key < pqueue.Elements[pqueue.A[s]].key)
        ):
            s = r
        if s != i:
            _exchange_nodes(pqueue, i, s)
            i = s
        else:
            break
    
cdef inline void _decrease_key_from_node_index(
    PriorityQueue* pqueue,
    size_t node_idx, 
    DTYPE_t key_new) noexcept nogil:
    """Decrease the key of an element in the priority queue, given its tree index.

    input
    =====
    * PriorityQueue* pqueue : priority queue
    * size_t node_idx : node index
    * DTYPE_t key_new : new key value

    assumptions
    ===========
    * pqueue.elements[pqueue.A[node_idx]] is in the heap (node_idx < pqueue.size)
    * key_new < pqueue.elements[pqueue.A[node_idx]].key
    """
    cdef:
        size_t i = node_idx, j
        DTYPE_t key_j
    pqueue.Elements[pqueue.A[i]].key = key_new
    while i > 0: 
        j = (i - 1) // 2  
        key_j = pqueue.Elements[pqueue.A[j]].key
        if key_j > key_new:
            _exchange_nodes(pqueue, i, j)
            i = j
        else:
            break

# Simple example
# ==============
cpdef test_01():
    cdef PriorityQueue pqueue
    init_pqueue(&pqueue, 4)
    insert(&pqueue, 1, 3.0)
    insert(&pqueue, 0, 2.0)
    insert(&pqueue, 3, 4.0)
    insert(&pqueue, 2, 1.0)
    assert pqueue.size == 4
    A_ref = [2, 0, 3, 1]
    n_ref = [1, 3, 0, 2]
    key_ref = [2.0, 3.0, 1.0, 4.0]
    for i in range(4):
        assert pqueue.A[i] == A_ref[i]
        assert pqueue.Elements[i].node_idx == n_ref[i]
        assert pqueue.Elements[i].state == IN_HEAP
        assert pqueue.Elements[i].key == key_ref[i]
    decrease_key(&pqueue, 3, 0.0)
    assert pqueue.size == 4
    A_ref = [3, 0, 2, 1]
    n_ref = [1, 3, 2, 0]
    key_ref = [2.0, 3.0, 1.0, 0.0]
    for i in range(4):
        assert pqueue.A[i] == A_ref[i]
        assert pqueue.Elements[i].node_idx == n_ref[i]
        assert pqueue.Elements[i].state == IN_HEAP
        assert pqueue.Elements[i].key == key_ref[i]
    
    element_idx = extract_min(&pqueue)
    assert element_idx == 3
    free_pqueue(&pqueue)


cpdef DTYPE_t[:, :] dijkstra_ref(
    DTYPE_t[:, :] cost,
    Py_ssize_t[:, :] targets,
):

    
    cdef:
        DTYPE_t _sqrt2 = np.sqrt(2)
        Py_ssize_t[8] NEIGHBOURS_X = [-1, 1, 0, 0, -1, 1, -1, 1]
        Py_ssize_t[8] NEIGHBOURS_Y = [0, 0, -1, 1, -1, -1, 1, 1]
        DTYPE_t[8] NEIGHBOURS_DISTANCE = [1, 1, 1, 1, _sqrt2, _sqrt2, _sqrt2, _sqrt2]
        Py_ssize_t width = cost.shape[0]
        Py_ssize_t height = cost.shape[1]
        DTYPE_t[:, :] dist = np.full_like(cost, np.inf)
        Py_ssize_t[:, :] prev_x = np.full_like(cost, -1, np.intp)
        Py_ssize_t[:, :] prev_y = np.full_like(cost, -1, np.intp)
        PriorityQueue q
        Py_ssize_t x, y, idx, x2, y2
        DTYPE_t alternative

    init_pqueue(&q, width * height)
    for i in range(targets.shape[0]):
        x = targets[i, 0]
        y = targets[i, 1]
        insert(&q, x + y * width, 0.0)
        dist[x, y] = 0.0

    while 0 < q.size:
        idx = extract_min(&q)
        x = idx % width
        y = idx // width
        for k in range(8):
            x2 = x + NEIGHBOURS_X[k]
            y2 = y + NEIGHBOURS_Y[k]
            alternative = dist[x, y] + NEIGHBOURS_DISTANCE[k] * cost[x2, y2]
            if alternative < dist[x2, y2]:
                dist[x2, y2] = alternative
                prev_x[x2, y2] = x
                prev_y[x2, y2] = y
                insert(&q, x2 + y2 * width, alternative)
                
    return dist
    
    
cdef class HeapElement:
    cdef Py_ssize_t x
    cdef Py_ssize_t y
    cdef DTYPE_t distance

    def __lt__(self, other):
        return True
        return self.distance < other.distance


cpdef DTYPE_t[:, :] dijkstra_heapq(
    DTYPE_t[:, :] cost,
    Py_ssize_t[:, :] targets,
):

    
    cdef:
        DTYPE_t _sqrt2 = np.sqrt(2)
        Py_ssize_t[8] NEIGHBOURS_X = [-1, 1, 0, 0, -1, 1, -1, 1]
        Py_ssize_t[8] NEIGHBOURS_Y = [0, 0, -1, 1, -1, -1, 1, 1]
        DTYPE_t[8] NEIGHBOURS_DISTANCE = [1, 1, 1, 1, _sqrt2, _sqrt2, _sqrt2, _sqrt2]
        Py_ssize_t width = cost.shape[0]
        Py_ssize_t height = cost.shape[1]
        DTYPE_t[:, :] dist = np.full_like(cost, np.inf)
        Py_ssize_t[:, :] prev_x = np.full_like(cost, -1, np.intp)
        Py_ssize_t[:, :] prev_y = np.full_like(cost, -1, np.intp)
        Py_ssize_t x, y, idx, x2, y2
        DTYPE_t alternative

    q: list[HeapElement] = []
    for i in range(targets.shape[0]):
        x = targets[i, 0]
        y = targets[i, 1]
        heapq.heappush(q, HeapElement(x, y, 0.0))
        dist[x, y] = 0.0

    while 0 < len(q):
        p = heapq.heappop(q)
        x = p.x
        y = p.y
        print(p, x, y)
        for k in range(8):
            x2 = x + NEIGHBOURS_X[k]
            y2 = y + NEIGHBOURS_Y[k]
            alternative = dist[x, y] + NEIGHBOURS_DISTANCE[k] * cost[x2, y2]
            if alternative < dist[x2, y2]:
                dist[x2, y2] = alternative
                prev_x[x2, y2] = x
                prev_y[x2, y2] = y
                heapq.heappush(q, HeapElement(x2, y2, alternative))
                
    return dist
    

cpdef Py_ssize_t[:] dijkstra_flat(
    DTYPE_t[:] cost,
    Py_ssize_t[:] targets,
    Py_ssize_t[:] neighbours,
):

    cdef:
        DTYPE_t[:] dist = np.full_like(cost, np.inf)
        Py_ssize_t[:] prev = np.full_like(cost, -1, np.intp)
        PriorityQueue q
        Py_ssize_t u, v, d, k
        DTYPE_t alternative

    init_pqueue(&q, len(cost))
    for u in targets:
        insert(&q, u, 0.0)
        dist[u] = 0.0

    while 0 < q.size:
        u = extract_min(&q)
        for k in range(4):
            v = u + neighbours[k]
            alternative = dist[u] + cost[v]
            if alternative < dist[v]:
                dist[v] = alternative
                prev[v] = k
                insert(&q, v, alternative)
                
    return prev

Content of stdout:
_cython_magic_d76df9e37423ba833c8e13799d22c3c8f829ae0d.c
   Creating library C:\Users\volke\.ipython\cython\Users\volke\.ipython\cython\_cython_magic_d76df9e37423ba833c8e13799d22c3c8f829ae0d.cp311-win_amd64.lib and object C:\Users\volke\.ipython\cython\Users\volke\.ipython\cython\_cython_magic_d76df9e37423ba833c8e13799d22c3c8f829ae0d.cp311-win_amd64.exp
Generating code
Finished generating codeContent of stderr:
cl : Command line warning D9025 : overriding '/Os' with '/Ot'
cl : Command line warning D9002 : ignoring unknown option '-Of'
cl : Command line warning D9002 : ignoring unknown option '-Oa'

In [81]:
cost = np.array(1 + scipy.stats.halfnorm.rvs(0, 1, size=game_info.map_size), dtype=np.float64)
cost = np.where(game_info.pathing_grid.data_numpy.T == 0, np.inf, cost)
px.imshow(cost)

In [82]:
targets = [tuple(np.random.randint((1, 1), np.subtract(game_info.map_size,  1))) for _ in range(25)]

In [83]:
px.imshow(dijkstra_ref(cost, np.array(targets, dtype=np.intp)))

In [99]:
%timeit dijkstra_ref(cost, np.array(targets, dtype=np.intp))

3.23 ms ± 143 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [98]:
%timeit dijkstra_heapq(cost, np.array(targets, dtype=np.intp))

AttributeError: '_cython_magic_d76df9e37423ba833c8e13799d22c3c8f829' object has no attribute 'x'

In [ ]:
%timeit shortest_paths_opt(cost, targets, diagonal=True)

In [ ]:
cost_flat = cost.flatten()
targets_flat = np.array([x * cost.shape[1] + y for x, y in targets], dtype=np.intp)
neighbours = np.array([-cost.shape[1], cost.shape[1], -1, 1], dtype=np.intp)

In [ ]:
def get_readonly_view(arr):
    result = arr.view()
    result.flags.writeable = False
    return result
def dijkstra_wrapper(
    cost: np.ndarray,
    targets: list[tuple[int, int]],
) -> np.ndarray:
    # cost_flat = cost.flatten()
    # targets_flat = np.array([x * cost.shape[1] + y for x, y in targets], dtype=np.intp)
    prev_flat = dijkstra(cost_flat, targets_flat, neighbours)
    return prev_flat
    # prev = get_readonly_view(prev_flat).reshape(cost.shape)
    # prev_x = prev // cost.shape[1]
    # prev_y = prev % cost.shape[1]
    # return np.dstack((prev_x, prev_y))

In [ ]:
cost_flat = cost.flatten()
targets_flat = np.array([x * cost.shape[1] + y for x, y in targets], dtype=np.intp)

In [ ]:
%%timeit
prev_flat = np.asarray(dijkstra(cost_flat, targets_flat, neighbours))
# prev_flat = np.ones_like(cost_flat, dtype=int)
prev = prev_flat.reshape(cost.shape)
# prev_x = prev // cost.shape[1]
# prev_y = prev % cost.shape[1]
# prev_xy = np.dstack((prev_x, prev_y))

In [ ]:
prev